### Importing Geometrical Information from Laser Scanners-

The module focuses on the ability to directly model real objects without having to enter their geometries manually. A laser scanner takes distance measurements in every direction to rapidly capture the surface shape of objects, buildings and landscapes.The data acquired from the laser scanner is in the form of point cloud. This information is then used to construct a full 3D model of the object in the form of a STL mesh. The point cloud data can be processed through various free mesh processing softwares (for example, MeshLab) to be converted into STL format. The STL mesh is then voxelized. A voxel is a unit of graphic information that defines a point in three-dimensional space. The voxelized model array is further used to build FDTD cells upon which gprMax is based. 

### GeometryObjectsReadSTL

This function allows the user to convert the STL models to voxelized mesh. The input parameter requires the user to specify the path of the STL file, a material index (an integer corresponding to the materials given in the materials.txt file that you want the model to be built of) and also the spacial discretization/resolution of the model.
<br>

Usage:
<br>
g_stl = gprMax.GeometryObjectsReadSTL(stl_file='your_model_file.stl', mat_index = n, discretization=(0.002,0.002,0.002))
<br>
where 'n' is an integer corresponding to the material
<br>
 
The voxelized mesh is exported in the form of a numpy 3D array and further written to a HDF5 file which is outputted. 
The HDF5 file along with the materials.txt file is given as input to the GeometryObjectsRead command which then builds the FDTD cells. The GeomteryView command is then used to output the model geometry which can be analysed/viewed in Paraview.
<br>

#### NOTE:
 - Before using this module make sure that your conda env has 'numpy-stl' pip dependency installed. You can check that by running 'conda list'. If not installed, then head over to the conda_env.yml file, under the pip section add 'numpy-stl' and rebuild the environment by using the following command
 <br>
 $ conda env update -f conda_env.yml

#### Resolution

Resolution is basically the no. of layers in which the model is divided. It can have different values for each axis, and these values are auto-calculated based on the spacial discretization entered by the user. A simple relation follows: smaller the spacial discretization, greater would be the resolution, thus greater would be the no. of voxels.




| STL Model of Mountain Topography |
|:----------------------------------|
|<img src='data_LasoMax/docs_images/mont_blanc_stl.png' height="250" width="350">|

| Voxelized Model of Topography with resolution 1mm | Voxelized Model of Topography with resolution 2mm |
|:---------------------------------------------------|:---------------------------------------------------|
|<img src='data_LasoMax/docs_images/mont_blanc.png' height="300" width="370">|<img src='data_LasoMax/docs_images/mont_blanc_2.png' height="300" width="370">|

In [ ]:
#Sample input file of Mont Blanc
from pathlib import Path

import gprMax

fn = Path('user_models/Mont_Blanc.py')

title = gprMax.Title(name=fn.with_suffix('').name)
domain = gprMax.Domain(p1=(0.4,0.4 ,0.12))
dxdydz = gprMax.Discretisation(p1=(0.002, 0.002, 0.002))
time_window = gprMax.TimeWindow(time=10e-9)

g_stl = gprMax.GeometryObjectsReadSTL(stl_file='data_LasoMax/Mont_Blanc.stl', mat_index=2, discretization=(0.002,0.002,0.002))

g_read = gprMax.GeometryObjectsRead(p1=(0.04,0.04,0.04), geofile=fn.with_suffix('.h5') , matfile='user_models/materials.txt')

gv = gprMax.GeometryView(p1=(0, 0, 0),
                         p2=(0.4, 0.4 , 0.12),
                         dl=(0.002, 0.002, 0.002),
                         filename=fn.with_suffix('').name,
                         output_type='n')
                         
# create a scene
scene = gprMax.Scene()
# add the simulation objects to the scene
scene.add(title)
scene.add(domain)
scene.add(dxdydz)
scene.add(time_window)
scene.add(g_stl)
scene.add(g_read)
scene.add(gv)

# run the simulation
gprMax.run(scenes=[scene], geometry_only=True, n=1, outputfile=fn)


|Some example STL models | Corresponding Voxelized Models |
|:------------------------|:--------------------------------|
|<img src='data_LasoMax/docs_images/bunny_stl.png' height="250" width="300">|<img src='data_LasoMax/docs_images/bunny_voxel.png' height="250" width="300">|
|<img src='data_LasoMax/docs_images/amphora_stl.png' height="300" width="300">|<img src='data_LasoMax/docs_images/amphora.png' height="300" width="300">|
|<img src='data_LasoMax/docs_images/house_stl.png' height="250" width="300">|<img src='data_LasoMax/docs_images/house.png' height="250" width="300">|
|<img src='data_LasoMax/docs_images/saint_michel_island_stl.png' height="250" width="300">|<img src='data_LasoMax/docs_images/saint_michel_island.png' height="250" width="300">|

### Point Cloud to STL

There are various free point cloud and mesh processing softwares available to convert laser point cloud data (in formats like .ply, .pcd, .xyz etc) to STL. Meshlab, which is one such software, is being used here. To convert your Point Cloud to STL,
firstly import your Point Cloud Data into Meshlab. If you feel the need you can scale your Point Cloud by heading over to 
Filter -> Normal and Curvature -> Scale and scale it accordingly.
If you wish to reduce the density of points in the cloud then under Remeshing use 'Quadric Edge Collapse Decimation'.
Finally export the mesh as STL. Now you are ready to import this STL file into gprMax using the GeometryObjectsReadSTL function.

| Point Cloud | Corresponding STL Mesh |
|:-------------|:------------------------|
|<img src='data_LasoMax/docs_images/alps_point_cloud.png' height="250" width="300">|<img src='data_LasoMax/docs_images/alp_stl.png' height="250" width="300">|